In [32]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.metrics.pairwise import cosine_similarity

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [33]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'amazon-beauty/CF_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/CF_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/CF_1_test_predictions_few_shot.csv


In [34]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rating          34 non-null     float64
 1   verified        34 non-null     bool   
 2   reviewTime      34 non-null     object 
 3   reviewerID      34 non-null     object 
 4   asin            34 non-null     object 
 5   reviewerName    34 non-null     object 
 6   reviewText      34 non-null     object 
 7   summary         34 non-null     object 
 8   unixReviewTime  34 non-null     object 
 9   vote            3 non-null      float64
 10  style           17 non-null     object 
 11  category        34 non-null     object 
 12  tech1           0 non-null      float64
 13  description     34 non-null     object 
 14  fit             0 non-null      float64
 15  title           34 non-null     object 
 16  also_buy        34 non-null     object 
 17  tech2           0 non-null      float

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
1,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
2,2.0,True,2018-03-27,A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,2018-03-27,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00
3,2.0,True,2018-03-27,A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,2018-03-27,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00
4,5.0,True,2017-02-02,A3A8F2URN7MEPR,B00006L9LC,Sheila T.,My favorite powder!,Five Stars,2017-02-02,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00


In [38]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Check if Target User Has Rated All Items
def check_unrated_items(user_item_matrix, user_id):
    return user_item_matrix.loc[user_id].isna().sum()


# Example function to create a user-item matrix
def create_user_item_matrix(data, user_column_name='reviewerID', item_column_name='title', rating_column_name='rating'):
    return pd.pivot_table(data, index=user_column_name, columns=item_column_name, values=rating_column_name)

# Example function for item-based collaborative filtering
def item_based_cf(user_item_matrix, target_user_id):
    # Fill NaN values with 0 (indicating no interaction)
    filled_user_item_matrix = user_item_matrix.fillna(0)

    # Transpose matrix to get item-user matrix
    item_user_matrix = filled_user_item_matrix.T

    # Compute item-item similarity matrix
    item_similarity = cosine_similarity(item_user_matrix, dense_output=True)

    # Convert to DataFrame
    item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

    # Get items rated by the target user
    target_user_ratings = filled_user_item_matrix.loc[target_user_id].dropna()

    # Compute weighted ratings
    weighted_ratings = item_similarity_df.loc[target_user_ratings.index].mul(target_user_ratings.values, axis=0)

    # Calculate recommendations
    recommendations = weighted_ratings.sum(axis=0).div(item_similarity_df.loc[target_user_ratings.index].sum(axis=0))
    recommendations = recommendations.drop(target_user_ratings.index, errors='ignore')  # Remove already rated items
    
    # Inside your item_based_cf function, add:
    print(f"Item Similarity DataFrame: {item_similarity_df.head()}")
    print(f"Target User Ratings: {target_user_ratings}")
    print(f"Weighted Ratings: {weighted_ratings.head()}")

    # Return top N recommendations
    return recommendations.nlargest(5)


# Example usage
user_item_matrix = create_user_item_matrix(data)
user_item_matrix

title,(Pack of 3) L.A. Colors Mineral Pressed Powder #Mp303 &quot;Creamy Natural&quot;,ABASSKY Lip Gloss Lipstick Matte Velvet Waterproof Super Long Lasting Not Fade (#18),"Astra Platinum Double Edge Safety Razor Blades ,100 Blades (20 x 5)","Avalon Grapefruit and Geranium Smoothing Shampoo, 11 Ounce",BIOAQUA 24K Gold Essence Collagen Skin Face Moisturizing Hyaluronic Acid Mask,Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz,Bonne Bell Smackers Bath and Body Starburst Collection,Charming Silver Colored Earring / Ear Cuff / Clip In Snake / Spiral Shape By VAGA,Citre Shine Moisture Burst Shampoo - 16 fl oz,"Dolce &amp; Gabbana Compact Parfum, 0.05 Ounce",...,"Oud Wood Oil. IMPRESSION of Tom Ford Oud Wood* Cologne for Men with SIMILAR Fragrance Accords, 10ml Amber Glass Roller, Black Cap; 100% Pure (Perfume Studio Oud Wood VERSION/TYPE; Not Original Brand)",Professional Waterproof Black Elegant EyeLiner Beauty Gel Mascara + Eyeliner Brush Makeup Tools,ROPALIA Cosmetic Makeup Waterproof Moisturize Lipstick Lip Stick Lip Gloss,"Reach Dentotape Waxed Dental Floss with Extra Wide Cleaning Surface for Large Spaces between Teeth, Unflavored, 100 Yards",Sankuwen 15PCs Wool Makeup Brush Set Tools Toiletry Kit,Urparcel Liquid Foundation Smooth Natural Long Lasting Makeup Beauty Cosmetics,Vintage Cinderella Pumpkin Carriage Locket Pendant Chain Necklace Xmas Gift,Vktech&reg; 20pcs Mix Colors Rhinestone Nose Studs Ring Bone Bar Pin Piercing Jewelry,Weixinbuy 1PC Secret Jelly Lipstick Color Changing Long Lasting Lip Gloss Random Color,"Zapzyt Maximum Strength 10% Benzoyl Peroxide Acne Treatment Gel, 1 Ounce"
reviewerID,,,,,,,,,,,,,,,,,,,,,
A1U8L4X1O2LXXF,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
A2TU781PWGS09X,NaN,NaN,NaN,2.0,NaN,2.0,2.0,NaN,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A38IPSLKOQ5O2F,NaN,4.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,3.0,NaN,5.0,NaN,3.0,5.0,5.0,NaN
A3A8F2URN7MEPR,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANV9L0JU6BNL,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0


In [39]:
unrated_items_count = check_unrated_items(user_item_matrix, 'ANV9L0JU6BNL')
print(f'Number of unrated items by user ANV9L0JU6BNL: {unrated_items_count}')
recommendations = item_based_cf(user_item_matrix, 'ANV9L0JU6BNL')
recommendations

Number of unrated items by user ANV9L0JU6BNL: 19
Item Similarity DataFrame: title                                               (Pack of 3) L.A. Colors Mineral Pressed Powder #Mp303 &quot;Creamy Natural&quot;  \
title                                                                                                                                  
(Pack of 3) L.A. Colors Mineral Pressed Powder ...                                           1.000000                                  
ABASSKY Lip Gloss Lipstick Matte Velvet Waterpr...                                           0.000000                                  
Astra Platinum Double Edge Safety Razor Blades ...                                           0.000000                                  
Avalon Grapefruit and Geranium Smoothing Shampo...                                           0.928477                                  
BIOAQUA 24K Gold Essence Collagen Skin Face Moi...                                           0.000000       

Series([], dtype: float64)